In [4]:
from pyspark import  SparkContext,SparkConf
sc = SparkContext( )
print (sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-1-ffd50910b931>:2 

In [ ]:
#load raw data and test data
raw_data = sc.textFile('/data/kddcup/kddcup.data_10_percent_corrected')
test_raw_data = sc.textFile('/data/kddcup/corrected')

In [22]:
#caculate the data lines
print raw_data.count()
print test_raw_data.count()

494021
311029


In [23]:
#get each feature 
print rawData.take(1)
raw_deal_data = raw_data.map(lambda x: x.split(","))
test_raw_deal_data = test_raw_data.map(lambda x: x.split(","))

[u'0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.']


In [94]:
#uniq 几个非数值类型，建立词典，后续将该特征转换为index
protocols = raw_deal_data.map(lambda x: x[1]).distinct().collect()
services = raw_deal_data.map(lambda x: x[2]).distinct().collect()
flags = raw_deal_data.map(lambda x: x[3]).distinct().collect()

print len(protocols)

3


In [43]:
from pyspark.mllib.regression import LabeledPoint
from numpy import array
def create_labeled_point(line_split):
    # leave_out = [41]
    clean_line_split = line_split[0:41]
    
    # convert protocol to numeric categorical variable
    try: 
        clean_line_split[1] = protocols.index(clean_line_split[1])
    except:
        clean_line_split[1] = len(protocols)
        
    # convert service to numeric categorical variable
    try:
        clean_line_split[2] = services.index(clean_line_split[2])
    except:
        clean_line_split[2] = len(services)
    
    # convert flag to numeric categorical variable
    try:
        clean_line_split[3] = flags.index(clean_line_split[3])
    except:
        clean_line_split[3] = len(flags)
    
    # convert label to binary label,normal=0.0, else 1.0
    attack = 1.0
    if line_split[41]=='normal.':
        attack = 0.0
   # print clean_line_split  
    return LabeledPoint(attack, array([float(x) for x in clean_line_split]))

In [81]:
training_data = raw_deal_data.map(create_labeled_point)
test_data = test_raw_deal_data.map(create_labeled_point)

In [82]:
#print training_data.take(5)

In [117]:
labels = raw_deal_data.map(lambda x: x[-1]).distinct()
print (labels.count())
print (labels.collect())
test_labels = test_raw_deal_data.map(lambda x:x[-1]).distinct()
print(test_labels.count())
print(test_labels.collect())

23
[u'phf.', u'imap.', u'normal.', u'spy.', u'back.', u'perl.', u'teardrop.', u'rootkit.', u'warezclient.', u'ftp_write.', u'land.', u'pod.', u'multihop.', u'neptune.', u'guess_passwd.', u'portsweep.', u'nmap.', u'loadmodule.', u'satan.', u'ipsweep.', u'buffer_overflow.', u'warezmaster.', u'smurf.']
38
[u'phf.', u'snmpgetattack.', u'xterm.', u'ps.', u'back.', u'mscan.', u'imap.', u'sendmail.', u'perl.', u'worm.', u'mailbomb.', u'apache2.', u'teardrop.', u'ftp_write.', u'xsnoop.', u'processtable.', u'rootkit.', u'sqlattack.', u'normal.', u'land.', u'portsweep.', u'buffer_overflow.', u'pod.', u'neptune.', u'guess_passwd.', u'snmpguess.', u'saint.', u'nmap.', u'loadmodule.', u'warezmaster.', u'udpstorm.', u'satan.', u'xlock.', u'ipsweep.', u'httptunnel.', u'named.', u'multihop.', u'smurf.']


In [95]:
#DecisionTree 2-classification
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from time import time

# Build the model
t0 = time()
tree_model = DecisionTree.trainClassifier(training_data, numClasses=2, \
                                          categoricalFeaturesInfo={1: len(protocols), 2: len(services), 3: len(flags)},\
                                          impurity='gini', maxDepth=4, maxBins=100)
tt = time() - t0

print "Classifier trained in {} seconds".format(round(tt,3))

Classifier trained in 22.914 seconds


In [118]:
t0 = time()
predictions = tree_model.predict(test_data.map(lambda p: p.features))
labels_and_preds = test_data.map(lambda p: p.label).zip(predictions)

test_accuracy = labels_and_preds.filter(lambda (v, p): v == p).count() / float(test_data.count())
tt = time() - t0
print labels_and_preds.take(5)

print "Prediction made in {} seconds. Test accuracy is {}".format(round(tt,3), round(test_accuracy,4))

[(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (1.0, 0.0), (1.0, 0.0)]
Prediction made in 17.153 seconds. Test accuracy is 0.9221


In [119]:
# Train a RandomForest model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
#  Note: Use larger numTrees in practice.
#  Setting featureSubsetStrategy="auto" lets the algorithm choose.
from pyspark.mllib.tree import RandomForest, RandomForestModel
numClasses=2
numTrees=10
rf_model = RandomForest.trainClassifier(training_data, numClasses=2, categoricalFeaturesInfo={},\
                                     numTrees=100, featureSubsetStrategy="auto",\
                                     impurity='gini',  maxDepth=4,   maxBins=32)


In [120]:
t0 = time()
predictions_rf = rf_model.predict(test_data.map(lambda p: p.features))
labels_and_preds_rf = test_data.map(lambda lp: lp.label).zip(predictions_rf)
#test_accuracy = labels_and_preds.filter(lambda (v, p): v == p).count() / float(test_data.count())
test_accuracy_rf = labels_and_preds_rf.filter(lambda (v, p): v == p).count() / float(test_data.count())
tt = time()-t0
print "Prediction made in {} seconds. Test accuracy is {}".format(round(tt,3), round(test_accuracy_rf,4))



Prediction made in 21.244 seconds. Test accuracy is 0.9215


In [116]:
# Save and load model
model.save(sc, "/data/kddcup/myRandomForestClassificationModel")
sameModel = RandomForestModel.load(sc, "/data/kddcup/myRandomForestClassificationModel")